In [1]:
import pandas as pd
import numpy as np
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
from datetime import datetime
import cv2
from deepface import DeepFace

from sklearn.metrics.pairwise import cosine_similarity

In [2]:
client_credentials_manager = SpotifyClientCredentials(client_id='ac1f41b0f93c45ac8af29623bdf94e0a', client_secret='f2460ff1a77e4c6db5eca633dde46d3b')
sp = spotipy.Spotify(client_credentials_manager = client_credentials_manager)


# playlist_link = "https://open.spotify.com/playlist/37i9dQZF1DX0XUfTFmNBRM?si=7934cecad3f3439c"
playlist_link = input("Enter the link: ")

playlist_URI = playlist_link.split("/")[-1].split("?")[0]

songs = sp.playlist_tracks(playlist_URI, market='IN')['items']  #will give a list of songs (meta-data)
# print(songs)

In [3]:
song_name = []
song_id = []
song_popu = []
song_added_date = []
song_release_date = []
for song in songs:
    song_name.append(song['track']['name'])
    song_id.append(song['track']['id'])
    song_popu.append(song['track']['popularity'])
    song_added_date.append(song['added_at'])
    song_release_date.append(song['track']['album']['release_date'])

# date_added = map(lambda x: int(x[5:7]), song_added_date)
# release_year = list(map(lambda x: datetime.strptime(x, '%Y-%m-%d').year, song_release_date))

In [4]:


# dt = datetime.strptime("2003-04-01", "%Y-%m-%d")
# dt

In [5]:
#combining the data
# df = df[df['danceability'] != '']
# print(df.iloc[40])
df = pd.DataFrame({
    'name':song_name,
    'popularity':song_popu,
    'date_added':pd.to_datetime(song_added_date),
    'release_year':list(map(lambda x: int(x[:4]), song_release_date))
    })

#audio features

features = sp.audio_features(song_id)
# print(features)
feat_names = list(sp.audio_features(song_id)[0].keys())

for row in range(len(features)):
    for col in range(len(feat_names)):
        df.loc[row, feat_names[col]] = features[row][feat_names[col]]
df.head()

,name,popularity,date_added,release_year,danceability,energy,key,loudness,mode,speechiness,...,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature
0,Natural,82,2023-01-17 11:35:57+00:00,2018,0.704,0.611,2.0,-6.112,1.0,0.0409,...,0.0812,0.220,100.000,audio_features,2FY7b99s15jUprqC0M5NCT,spotify:track:2FY7b99s15jUprqC0M5NCT,https://api.spotify.com/v1/tracks/2FY7b99s15jU...,https://api.spotify.com/v1/audio-analysis/2FY7...,189467.0,4.0
1,Overwhelmed (Ryan Mack Remix),77,2023-01-17 11:35:57+00:00,2021,0.658,0.445,0.0,-8.749,0.0,0.1540,...,0.0845,0.513,174.111,audio_features,03eyNjBM2mpx28H6kdaufN,spotify:track:03eyNjBM2mpx28H6kdaufN,https://api.spotify.com/v1/tracks/03eyNjBM2mpx...,https://api.spotify.com/v1/audio-analysis/03ey...,103240.0,4.0
2,Left and Right (Feat. Jung Kook of BTS),82,2023-01-17 11:35:57+00:00,2022,0.881,0.592,2.0,-4.898,1.0,0.0324,...,0.0901,0.719,101.058,audio_features,5Odq8ohlgIbQKMZivbWkEo,spotify:track:5Odq8ohlgIbQKMZivbWkEo,https://api.spotify.com/v1/tracks/5Odq8ohlgIbQ...,https://api.spotify.com/v1/audio-analysis/5Odq...,154487.0,4.0
3,Shivers,87,2023-01-17 11:35:57+00:00,2021,0.788,0.859,2.0,-2.724,1.0,0.0856,...,0.0424,0.822,141.020,audio_features,50nfwKoDiSYg8zOCREWAm5,spotify:track:50nfwKoDiSYg8zOCREWAm5,https://api.spotify.com/v1/tracks/50nfwKoDiSYg...,https://api.spotify.com/v1/audio-analysis/50nf...,207853.0,4.0
4,On My Way,71,2023-01-17 11:35:57+00:00,2021,0.522,0.697,1.0,-5.288,0.0,0.1030,...,0.3190,0.235,169.775,audio_features,3BomnHfqKZCdPJC9bkGBjE,spotify:track:3BomnHfqKZCdPJC9bkGBjE,https://api.spotify.com/v1/tracks/3BomnHfqKZCd...,https://api.spotify.com/v1/audio-analysis/3Bom...,193959.0,4.0


In [6]:
artists_col = []
for i in range(len(songs)):
    all_artists = songs[i]['track']['artists']
    artists = []
    for a in all_artists:
        artists.append(a['name'])
    artists_col.append(artists)
df['artists'] = artists_col

In [7]:
# print(df['artists'][2])
# print(df['artists'][2][1])
# df['date_added'][0]

In [8]:
# print(type(date[0]))

curr_month = datetime.today().month
curr_year = datetime.today().year

# year = int(x[:4])
# month = int(x[5:7])
recency = list(map(lambda x: curr_month - x.month if (x.year == curr_year) else curr_month + (12 - x.month)
                    + (curr_year - x.year - 1) * 12, df['date_added']))
df['recency'] = recency

In [9]:
df.head()

,name,popularity,date_added,release_year,danceability,energy,key,loudness,mode,speechiness,...,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature,artists,recency
0,Natural,82,2023-01-17 11:35:57+00:00,2018,0.704,0.611,2.0,-6.112,1.0,0.0409,...,100.000,audio_features,2FY7b99s15jUprqC0M5NCT,spotify:track:2FY7b99s15jUprqC0M5NCT,https://api.spotify.com/v1/tracks/2FY7b99s15jU...,https://api.spotify.com/v1/audio-analysis/2FY7...,189467.0,4.0,[Imagine Dragons],4
1,Overwhelmed (Ryan Mack Remix),77,2023-01-17 11:35:57+00:00,2021,0.658,0.445,0.0,-8.749,0.0,0.1540,...,174.111,audio_features,03eyNjBM2mpx28H6kdaufN,spotify:track:03eyNjBM2mpx28H6kdaufN,https://api.spotify.com/v1/tracks/03eyNjBM2mpx...,https://api.spotify.com/v1/audio-analysis/03ey...,103240.0,4.0,[Ryan Mack],4
2,Left and Right (Feat. Jung Kook of BTS),82,2023-01-17 11:35:57+00:00,2022,0.881,0.592,2.0,-4.898,1.0,0.0324,...,101.058,audio_features,5Odq8ohlgIbQKMZivbWkEo,spotify:track:5Odq8ohlgIbQKMZivbWkEo,https://api.spotify.com/v1/tracks/5Odq8ohlgIbQ...,https://api.spotify.com/v1/audio-analysis/5Odq...,154487.0,4.0,"[Charlie Puth, Jung Kook, BTS]",4
3,Shivers,87,2023-01-17 11:35:57+00:00,2021,0.788,0.859,2.0,-2.724,1.0,0.0856,...,141.020,audio_features,50nfwKoDiSYg8zOCREWAm5,spotify:track:50nfwKoDiSYg8zOCREWAm5,https://api.spotify.com/v1/tracks/50nfwKoDiSYg...,https://api.spotify.com/v1/audio-analysis/50nf...,207853.0,4.0,[Ed Sheeran],4
4,On My Way,71,2023-01-17 11:35:57+00:00,2021,0.522,0.697,1.0,-5.288,0.0,0.1030,...,169.775,audio_features,3BomnHfqKZCdPJC9bkGBjE,spotify:track:3BomnHfqKZCdPJC9bkGBjE,https://api.spotify.com/v1/tracks/3BomnHfqKZCd...,https://api.spotify.com/v1/audio-analysis/3Bom...,193959.0,4.0,"[Alan Walker, Sabrina Carpenter, Farruko]",4


In [10]:
#normalizing popularity before ohe

# max(df['popularity'])
# popu ranges between 0 to 100, so normalizing it to 0 to 20
df['popularity'] = list(map(lambda x: x // 5, df['popularity']))
# df.head(20)
df.head()

,name,popularity,date_added,release_year,danceability,energy,key,loudness,mode,speechiness,...,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature,artists,recency
0,Natural,16,2023-01-17 11:35:57+00:00,2018,0.704,0.611,2.0,-6.112,1.0,0.0409,...,100.000,audio_features,2FY7b99s15jUprqC0M5NCT,spotify:track:2FY7b99s15jUprqC0M5NCT,https://api.spotify.com/v1/tracks/2FY7b99s15jU...,https://api.spotify.com/v1/audio-analysis/2FY7...,189467.0,4.0,[Imagine Dragons],4
1,Overwhelmed (Ryan Mack Remix),15,2023-01-17 11:35:57+00:00,2021,0.658,0.445,0.0,-8.749,0.0,0.1540,...,174.111,audio_features,03eyNjBM2mpx28H6kdaufN,spotify:track:03eyNjBM2mpx28H6kdaufN,https://api.spotify.com/v1/tracks/03eyNjBM2mpx...,https://api.spotify.com/v1/audio-analysis/03ey...,103240.0,4.0,[Ryan Mack],4
2,Left and Right (Feat. Jung Kook of BTS),16,2023-01-17 11:35:57+00:00,2022,0.881,0.592,2.0,-4.898,1.0,0.0324,...,101.058,audio_features,5Odq8ohlgIbQKMZivbWkEo,spotify:track:5Odq8ohlgIbQKMZivbWkEo,https://api.spotify.com/v1/tracks/5Odq8ohlgIbQ...,https://api.spotify.com/v1/audio-analysis/5Odq...,154487.0,4.0,"[Charlie Puth, Jung Kook, BTS]",4
3,Shivers,17,2023-01-17 11:35:57+00:00,2021,0.788,0.859,2.0,-2.724,1.0,0.0856,...,141.020,audio_features,50nfwKoDiSYg8zOCREWAm5,spotify:track:50nfwKoDiSYg8zOCREWAm5,https://api.spotify.com/v1/tracks/50nfwKoDiSYg...,https://api.spotify.com/v1/audio-analysis/50nf...,207853.0,4.0,[Ed Sheeran],4
4,On My Way,14,2023-01-17 11:35:57+00:00,2021,0.522,0.697,1.0,-5.288,0.0,0.1030,...,169.775,audio_features,3BomnHfqKZCdPJC9bkGBjE,spotify:track:3BomnHfqKZCdPJC9bkGBjE,https://api.spotify.com/v1/tracks/3BomnHfqKZCd...,https://api.spotify.com/v1/audio-analysis/3Bom...,193959.0,4.0,"[Alan Walker, Sabrina Carpenter, Farruko]",4


In [11]:
for i in range(len(df['release_year'])):
    if df.loc[i, 'release_year'] == 0:
        df.drop(i, inplace=True)

In [12]:
#one-hot-encoding

# pd.get_dummies(df['release_year']).head(10)
pd.get_dummies(df['popularity']).head()

,0,8,9,10,11,12,13,14,15,16,17,18
0,0,0,0,0,0,0,0,0,0,1,0,0
1,0,0,0,0,0,0,0,0,1,0,0,0
2,0,0,0,0,0,0,0,0,0,1,0,0
3,0,0,0,0,0,0,0,0,0,0,1,0
4,0,0,0,0,0,0,0,1,0,0,0,0


In [13]:
# for cosin sim, we need the size of the vectors to be same, so we are genralizing the columns

for i in range(0,21):
    df[f"popu|{i}"] = [0] * len(df['name'])

for i in range(1980, datetime.today().year):
    df[f"year|{i}"] = [0] * len(df['name'])

df.head()

,name,popularity,date_added,release_year,danceability,energy,key,loudness,mode,speechiness,...,year|2013,year|2014,year|2015,year|2016,year|2017,year|2018,year|2019,year|2020,year|2021,year|2022
0,Natural,16,2023-01-17 11:35:57+00:00,2018,0.704,0.611,2.0,-6.112,1.0,0.0409,...,0,0,0,0,0,0,0,0,0,0
1,Overwhelmed (Ryan Mack Remix),15,2023-01-17 11:35:57+00:00,2021,0.658,0.445,0.0,-8.749,0.0,0.1540,...,0,0,0,0,0,0,0,0,0,0
2,Left and Right (Feat. Jung Kook of BTS),16,2023-01-17 11:35:57+00:00,2022,0.881,0.592,2.0,-4.898,1.0,0.0324,...,0,0,0,0,0,0,0,0,0,0
3,Shivers,17,2023-01-17 11:35:57+00:00,2021,0.788,0.859,2.0,-2.724,1.0,0.0856,...,0,0,0,0,0,0,0,0,0,0
4,On My Way,14,2023-01-17 11:35:57+00:00,2021,0.522,0.697,1.0,-5.288,0.0,0.1030,...,0,0,0,0,0,0,0,0,0,0


In [14]:
df_year = pd.get_dummies(df['release_year'])
df_popu = pd.get_dummies(df['popularity'])
df_year.columns = map(lambda x: 'year' + '|' + str(x), df_year.columns)
df_popu.columns = map(lambda x: 'popu' + '|' + str(x), df_popu.columns)
df_popu.head()

,popu|0,popu|8,popu|9,popu|10,popu|11,popu|12,popu|13,popu|14,popu|15,popu|16,popu|17,popu|18
0,0,0,0,0,0,0,0,0,0,1,0,0
1,0,0,0,0,0,0,0,0,1,0,0,0
2,0,0,0,0,0,0,0,0,0,1,0,0
3,0,0,0,0,0,0,0,0,0,0,1,0
4,0,0,0,0,0,0,0,1,0,0,0,0


In [15]:
#now updating the columns with values wherever needed

for col in df_popu.columns:
    df[col] = df_popu[col]
for col in df_year.columns:
    df[col] = df_year[col]

In [16]:
df.columns

Index(['name', 'popularity', 'date_added', 'release_year', 'danceability',
       'energy', 'key', 'loudness', 'mode', 'speechiness', 'acousticness',
       'instrumentalness', 'liveness', 'valence', 'tempo', 'type', 'id', 'uri',
       'track_href', 'analysis_url', 'duration_ms', 'time_signature',
       'artists', 'recency', 'popu|0', 'popu|1', 'popu|2', 'popu|3', 'popu|4',
       'popu|5', 'popu|6', 'popu|7', 'popu|8', 'popu|9', 'popu|10', 'popu|11',
       'popu|12', 'popu|13', 'popu|14', 'popu|15', 'popu|16', 'popu|17',
       'popu|18', 'popu|19', 'popu|20', 'year|1980', 'year|1981', 'year|1982',
       'year|1983', 'year|1984', 'year|1985', 'year|1986', 'year|1987',
       'year|1988', 'year|1989', 'year|1990', 'year|1991', 'year|1992',
       'year|1993', 'year|1994', 'year|1995', 'year|1996', 'year|1997',
       'year|1998', 'year|1999', 'year|2000', 'year|2001', 'year|2002',
       'year|2003', 'year|2004', 'year|2005', 'year|2006', 'year|2007',
       'year|2008', 'year|2009

In [17]:
df.iloc[:0, :50]

,name,popularity,date_added,release_year,danceability,energy,key,loudness,mode,speechiness,...,popu|16,popu|17,popu|18,popu|19,popu|20,year|1980,year|1981,year|1982,year|1983,year|1984


In [18]:
#explode actually seperates the rows according to the list in refered column cells
# pd.get_dummies(df['artists'])

In [19]:
exploded_df = df.explode('artists')['artists'].reset_index().drop('index', axis=1)
exploded_df.head()

,artists
0,Imagine Dragons
1,Ryan Mack
2,Charlie Puth
3,Jung Kook
4,BTS


In [20]:
artists_excel = pd.read_excel('datasets/artists_names.xlsx')
# artists_excel['artists']
# for name in artists_excel['artists']:
#     # exploded_df = exploded_df.assign( = [0] * len(exploded_df['artists']))
#     exploded_df[f"artist|{name}"] = [0] * len(exploded_df['artists'])



# exploded_df = exploded_df.copy()

In [21]:
exploded_df

,artists
0,Imagine Dragons
1,Ryan Mack
2,Charlie Puth
3,Jung Kook
4,BTS
...,...
78,Alan Walker
79,Jamie Miller
80,Avicii
81,Glass Animals


In [23]:
x = df

In [24]:
# df_art = pd.get_dummies(exploded_df['artists'])
# print(exploded_df.columns)
new_df = df.copy()

zeros = [0] * len(new_df['name'])
extra = pd.DataFrame(zeros)
for name in artists_excel['artists']:
    extra[f"artist|{name}"] = 0

new_df = pd.concat([new_df, extra], axis=1)
new_df.dropna(axis=0, inplace=True)
for i, row in new_df.iterrows():
    print(row['artists'])
    for name in row['artists']:
        # print(name)
        # print(artists_excel['artists'])
        if name in list(artists_excel['artists']):
            new_df.loc[i, f"artist|{name}"] = 1


new_df = new_df.drop(0, axis=1)

# new_df.to_excel("test.xlsx")

# df_art.columns = map(lambda x: 'artists' + '|' + str(x), df_art.columns)

# for col in exploded_df.columns:
#     if col in df_art.columns:
#         exploded_df[col] = df_art[col]
# exploded_df

['Imagine Dragons']
['Ryan Mack']
['Charlie Puth', 'Jung Kook', 'BTS']
['Ed Sheeran']
['Alan Walker', 'Sabrina Carpenter', 'Farruko']
['Nicky Youre', 'dazy']
['The Kid LAROI', 'Justin Bieber']
['Imagine Dragons']
['salem ilese', 'TOMORROW X TOGETHER', 'Alan Walker']
['Justin Bieber']
['Ed Sheeran', 'Armaan Malik']
['Ryan Mack']
['Zedd', 'Elley Duhé']
['OneRepublic']
['Jazmine Sullivan']
['salem ilese']
['Justin Bieber', 'Daniel Caesar', 'Giveon']
['Ed Sheeran']
['NIKI DEMAR']
['Shawn Mendes']
['Charlie Puth', 'Selena Gomez']
['Imagine Dragons']
['Reagan Beem']
['Camila Cabello', 'Ed Sheeran']
['Ryan Mack']
['Conor Maynard']
['Shawn Mendes']
['Charlie Puth']
['The Weeknd']
['OneRepublic']
['Ed Sheeran']
['AJ Mitchell']
['Conor Maynard']
['Shawn Mendes', 'Camila Cabello']
['Ryan Mack']
['Lost Frequencies', 'Elley Duhé', 'X Ambassadors']
['GAYLE']
['Ed Sheeran', 'Justin Bieber']
['Imagine Dragons', 'Miss Congeniality']
['Torpe', 'Baby Tyson', 'Seb Tik']
['Shawn Mendes']
['Ryan Mack']
['On

In [25]:
new_df = new_df.copy()


In [26]:
# new df for generating the recommedation vector
# we are dropping the non-integer columns as they are of no use in calulating the similarity
recomm_vec_df = new_df.drop(['name', 'popularity', 'date_added', 'release_year', 'type', 'id', 'uri', 'track_href',  'analysis_url', 'artists'], axis=1)
recomm_vec_df.columns

Index(['danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness',
       'acousticness', 'instrumentalness', 'liveness', 'valence',
       ...
       'artist|Lost Frequencies', 'artist|Miss Congeniality', 'artist|Torpe',
       'artist|Baby Tyson', 'artist|Seb Tik', 'artist|Confetti',
       'artist|Victoria Justice', 'artist|Tainy', 'artist|Glee Cast',
       'artist|Jamie Miller'],
      dtype='object', length=1309)

In [27]:
# now calculating the weights which are going to be multiplied with each of the rows individually.
# for that we need to understand this that the bias must reduce the values of the older songs, so we need bias
# to be between 0 and 1
# 1 / recency is not working as it is drastically reducing the values whih can negatively impact the recommendations
# 0.9 ** recency might work. For recency 3, we get the weight as 0.729, which is totally fine as we tend to listen less
# to songs which are older than 3 months. Also, going below this value can trigger false recommendations
# through this we are actually reducing the effect of older (added) songs

recomm_vec_df['bias'] = list(map(lambda x: round(0.9 ** x, 5), list(df["recency"])))
for col in recomm_vec_df.columns:
    if col != 'bias':
        recomm_vec_df[col] = recomm_vec_df[col] * recomm_vec_df['bias']
recomm_vec_df.head(10)

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,...,artist|Miss Congeniality,artist|Torpe,artist|Baby Tyson,artist|Seb Tik,artist|Confetti,artist|Victoria Justice,artist|Tainy,artist|Glee Cast,artist|Jamie Miller,weights
0,0.461894,0.400877,1.3122,-4.010083,0.6561,0.026834,0.142374,0.000000,0.053275,0.144342,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.6561
1,0.431714,0.291965,0.0000,-5.740219,0.0000,0.101039,0.105632,0.000000,0.055440,0.336579,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.6561
2,0.578024,0.388411,1.3122,-3.213578,0.6561,0.021258,0.406126,0.000009,0.059115,0.471736,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.6561
3,0.517007,0.563590,1.3122,-1.787216,0.6561,0.056162,0.184364,0.000000,0.027819,0.539314,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.6561
4,0.342484,0.457302,0.6561,-3.469457,0.0000,0.067578,0.019093,0.000000,0.209296,0.154184,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.6561
5,0.503885,0.468455,6.5610,-3.352671,0.6561,0.026310,0.230947,0.000000,0.098415,0.552436,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.6561
6,0.387755,0.501260,0.6561,-3.598052,0.6561,0.031690,0.025129,0.000000,0.067578,0.313616,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.6561
7,0.506509,0.492075,3.2805,-2.407887,0.0000,0.029853,0.013188,0.000000,0.048551,0.385131,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.6561
8,0.394972,0.421216,1.3122,-3.545564,0.0000,0.138437,0.008595,0.000003,0.043827,0.354294,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.6561
9,0.431714,0.415967,0.6561,-3.803412,0.0000,0.027097,0.006955,0.000000,0.086605,0.190269,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.6561


In [52]:
list(recomm_vec_df.columns)

['danceability',
 'energy',
 'key',
 'loudness',
 'mode',
 'speechiness',
 'acousticness',
 'instrumentalness',
 'liveness',
 'valence',
 'tempo',
 'duration_ms',
 'time_signature',
 'popu|0',
 'popu|1',
 'popu|2',
 'popu|3',
 'popu|4',
 'popu|5',
 'popu|6',
 'popu|7',
 'popu|8',
 'popu|9',
 'popu|10',
 'popu|11',
 'popu|12',
 'popu|13',
 'popu|14',
 'popu|15',
 'popu|16',
 'popu|17',
 'popu|18',
 'popu|19',
 'popu|20',
 'year|1980',
 'year|1981',
 'year|1982',
 'year|1983',
 'year|1984',
 'year|1985',
 'year|1986',
 'year|1987',
 'year|1988',
 'year|1989',
 'year|1990',
 'year|1991',
 'year|1992',
 'year|1993',
 'year|1994',
 'year|1995',
 'year|1996',
 'year|1997',
 'year|1998',
 'year|1999',
 'year|2000',
 'year|2001',
 'year|2002',
 'year|2003',
 'year|2004',
 'year|2005',
 'year|2006',
 'year|2007',
 'year|2008',
 'year|2009',
 'year|2010',
 'year|2011',
 'year|2012',
 'year|2013',
 'year|2014',
 'year|2015',
 'year|2016',
 'year|2017',
 'year|2018',
 'year|2019',
 'year|2020',
 '

In [29]:
recomm_vec_df = recomm_vec_df.dropna().drop(['bias', 'recency'], axis=1)

In [30]:
recomm_vec_df.columns[:80]  #1191 columns
# new_df['artists']

Index(['danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness',
       'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo',
       'duration_ms', 'time_signature', 'popu|0', 'popu|1', 'popu|2', 'popu|3',
       'popu|4', 'popu|5', 'popu|6', 'popu|7', 'popu|8', 'popu|9', 'popu|10',
       'popu|11', 'popu|12', 'popu|13', 'popu|14', 'popu|15', 'popu|16',
       'popu|17', 'popu|18', 'popu|19', 'popu|20', 'year|1980', 'year|1981',
       'year|1982', 'year|1983', 'year|1984', 'year|1985', 'year|1986',
       'year|1987', 'year|1988', 'year|1989', 'year|1990', 'year|1991',
       'year|1992', 'year|1993', 'year|1994', 'year|1995', 'year|1996',
       'year|1997', 'year|1998', 'year|1999', 'year|2000', 'year|2001',
       'year|2002', 'year|2003', 'year|2004', 'year|2005', 'year|2006',
       'year|2007', 'year|2008', 'year|2009', 'year|2010', 'year|2011',
       'year|2012', 'year|2013', 'year|2014', 'year|2015', 'year|2016',
       'year|2017', 'year|2018', 'year

In [31]:
recomm_vec1 = np.array(list(map(lambda col: recomm_vec_df[col].mean(), recomm_vec_df.loc[:, :f"year|{datetime.today().year}"].columns)))
recomm_vec2 = np.array(list(map(lambda col: sum(recomm_vec_df[col]), recomm_vec_df.iloc[:, -len(artists_excel['artists']):].columns)))
# recomm_vec_df.loc[0, :f'year|{datetime.today().year}']
# recomm_vec_df.iloc[:, -len(artists_excel['artists']):]

In [32]:
# recomm_vec_df.iloc[:, -len(artists_excel['artists']):]
list(recomm_vec_df.loc[:, :f"year|{datetime.today().year}"].columns)

['danceability',
 'energy',
 'key',
 'loudness',
 'mode',
 'speechiness',
 'acousticness',
 'instrumentalness',
 'liveness',
 'valence',
 'tempo',
 'duration_ms',
 'time_signature',
 'popu|0',
 'popu|1',
 'popu|2',
 'popu|3',
 'popu|4',
 'popu|5',
 'popu|6',
 'popu|7',
 'popu|8',
 'popu|9',
 'popu|10',
 'popu|11',
 'popu|12',
 'popu|13',
 'popu|14',
 'popu|15',
 'popu|16',
 'popu|17',
 'popu|18',
 'popu|19',
 'popu|20',
 'year|1980',
 'year|1981',
 'year|1982',
 'year|1983',
 'year|1984',
 'year|1985',
 'year|1986',
 'year|1987',
 'year|1988',
 'year|1989',
 'year|1990',
 'year|1991',
 'year|1992',
 'year|1993',
 'year|1994',
 'year|1995',
 'year|1996',
 'year|1997',
 'year|1998',
 'year|1999',
 'year|2000',
 'year|2001',
 'year|2002',
 'year|2003',
 'year|2004',
 'year|2005',
 'year|2006',
 'year|2007',
 'year|2008',
 'year|2009',
 'year|2010',
 'year|2011',
 'year|2012',
 'year|2013',
 'year|2014',
 'year|2015',
 'year|2016',
 'year|2017',
 'year|2018',
 'year|2019',
 'year|2020',
 '

In [33]:
data_ = pd.read_excel('datasets/final_data.xlsx')

In [ ]:
#emotion code
# importing libraries
   




face_cascade = cv2.CascadeClassifier('haarcascade.xml')

# print(face_cascade)

cap = cv2.VideoCapture(0)
while True:
    ret,frame = cap.read()
    #print(frame)
    result = DeepFace.analyze(img_path = frame , actions=['emotion'], enforce_detection=False )
    #print(actions)
    gray = cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)

    faces = face_cascade.detectMultiScale(gray,1.1,4)

    for (x,y,w,h) in faces:
        cv2.rectangle(frame,(x,y),(x+w,y+h),(255,0,0),3)

    # emotion = result['dominant_emotion']
    emotion = result[0]['dominant_emotion']
    #print(emotion)

    txt = str(emotion)

    cv2.putText(frame,txt,(50,50),cv2.FONT_HERSHEY_SIMPLEX,1,(0,0,255),3)
    cv2.imshow('frame',frame)

    if cv2.waitKey(1) & 0xff == ord('q'):
        break



print(emotion)
cap.release()
cv2.destroyAllWindows()
#output: emotion


Action: emotion:   0%|          | 0/1 [00:00<?, ?it/s]


KeyboardInterrupt: 

In [34]:
data = data_.copy()

In [35]:
filt_col = ['acousticness', 'danceability', 'energy', 'loudness', 'tempo', 'valence']

happy_low = [0, 0.57, 0.4, -10.4, 75 ,0.25]
sad_low = [0.15, 0.3, 0.27, -4, 70, 0]
chill_low = [0, 0.3, 0.2, -15, 80, 0.2]
angry_low = [0, 0.46, 0.56, -11, 90, 0.1]

happy_high = [0.75, 0.86, 1, -3, 150, 1]
sad_high = [0.9, 0.7, 0.7, -12, 170, 0.7]
chill_high = [1, 0.8, 0.8, -4, 180, 0.9]
angry_high = [0.6, 0.85, 1, -4, 170, 0.8]

In [36]:
emotion = 'happy'

In [37]:
i = 0
if emotion == 'happy':
    for col in filt_col:
        data = data[(data[col] > happy_low[i]) & (data[col] < happy_high[i])]
        i += 1

elif emotion == 'sad':
    print(data.head())
    for col in filt_col:
        data = data[(data[col] > sad_low[i]) & (data[col] < sad_high[i])]
        i += 1

elif emotion == 'neutral':
    for col in filt_col:
        data = data[(data[col] > chill_low[i]) & (data[col] < chill_high[i])]
        i += 1

elif emotion == 'angry':
    for col in filt_col:
        data = data[(data[col] > angry_low[i]) & (data[col] < angry_high[i])]
        i += 1

In [38]:
data_filtered = data.drop(['name', 'popularity', 'date_added', 'release_year', 'type', 'id', 'uri', 'track_href',  'analysis_url', 'artists', 'Unnamed: 0'], axis=1)

In [39]:
data_filtered.iloc[:, -len(artists_excel['artists']) :].describe()

,artist|Iqlipse Nova,artist|Aditya A,artist|KASYAP,artist|The Yellow Diary,artist|Dikshant,artist|Akshath,artist|Panther,artist|Ankee,artist|Shilpa Rao,artist|Tech Panda,...,artist|Lost Frequencies,artist|Miss Congeniality,artist|Torpe,artist|Baby Tyson,artist|Seb Tik,artist|Confetti,artist|Victoria Justice,artist|Tainy,artist|Glee Cast,artist|Jamie Miller
count,158.0,158.0,158.0,158.0,158.0,158.0,158.0,158.0,158.000000,158.0,...,158.0,158.0,158.0,158.0,158.0,158.0,158.0,158.0,158.0,158.0
mean,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.012658,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
std,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.112150,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
min,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
25%,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
50%,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
75%,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
max,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [40]:
data_filtered.drop(0, axis=1, inplace=True)

In [41]:
# d = dict(data_filtered.isnull().sum())
# for key in d.keys():
#     if d[key] > 0:
#         print(key)

# data_filtered = data_filtered.drop()

In [42]:
np.isnan(recomm_vec2)
np.count_nonzero(np.isnan(recomm_vec2))

0

In [43]:
# dict(data.iloc[8])
list(recomm_vec1)
# len(recomm_vec1)

[0.43334311500000006,
 0.40063653,
 3.225825,
 -4.081018545,
 0.40459499999999987,
 0.049039100999999995,
 0.178441386885,
 6.380583434999998e-05,
 0.1160433135,
 0.34616382749999997,
 79.31302029,
 119821.70592000001,
 2.5915950000000003,
 0.010935,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.04374,
 0.032805,
 0.02187,
 0.032805,
 0.032805,
 0.054675,
 0.076545,
 0.076545,
 0.120285,
 0.13122,
 0.02187,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.010935,
 0.010935,
 0.0,
 0.04374,
 0.06561,
 0.02187,
 0.04374,
 0.07654500000000002,
 0.229635,
 0.14215499999999998,
 0.010935]

In [44]:
l1 = []
l2 = []
s = 0
recommendations = pd.DataFrame({'name': data['name'], 'id': data['id'], 'link':data['track_href']})
for i in range(len(data_filtered)):
    data_1 = data_filtered.loc[:, : f'year|{datetime.today().year}'].iloc[i].values
    data_2 = data_filtered.iloc[:, -len(artists_excel['artists']) :].iloc[i].values
    cos1 = cosine_similarity([recomm_vec1], [data_1])
    # print(cos1)

    if np.dot(recomm_vec2, data_2) != 0:
        cos2 = cosine_similarity([recomm_vec2], [data_2])
    else:
        cos2 = [[0]]  #this was done to avoid 0/0 situation which was happening when all cells of the data_2 were 0
        #by making cos2 0, we are nullifying its contribution
    
    l1.append(round(cos1[0][0], 10))
    l2.append(round(cos2[0][0], 10))
    # print("vector: ", np.isnan(recomm_vec2))
    # print("val: ", cos2)
    # print("data: ", np.count_nonzero(np.isnan(data_2)))

score = np.array(l1) + np.array(l2)
# print(score.astype(str))
recommendations['cosin'] = score.astype(str)
recommendations = recommendations.sort_values('cosin', ascending=False)
recommendations

,name,id,link,cosin
26,"Raataan Lambiyan (From ""Shershaah"")",2rOnSn2piaqLAlYjtfUBlY,https://api.spotify.com/v1/tracks/2rOnSn2piaqL...,1.1097642117
216,Raataan Lambiyan,1stiSonuKkZqhI1o9nZ9MT,https://api.spotify.com/v1/tracks/1stiSonuKkZq...,1.1097642117
55,Dil Galti Kar Baitha Hai,7HPM13Xek5XoRWxRzHfvkB,https://api.spotify.com/v1/tracks/7HPM13Xek5Xo...,1.1097642067
83,Without Me,6FZDfxM3a3UCqtzo5pxSLZ,https://api.spotify.com/v1/tracks/6FZDfxM3a3UC...,0.9999999999
65,Feather,2Zo1PcszsT9WQ0ANntJbID,https://api.spotify.com/v1/tracks/2Zo1PcszsT9W...,0.9999999999
...,...,...,...,...
329,Mast Magan,3uL1IBFhg52VcQqOwAG01E,https://api.spotify.com/v1/tracks/3uL1IBFhg52V...,0.9999999331
265,Mast Magan,3uL1IBFhg52VcQqOwAG01E,https://api.spotify.com/v1/tracks/3uL1IBFhg52V...,0.9999999331
118,Un Poco Loco,0OFknyqxmSQ42SoKxWVTok,https://api.spotify.com/v1/tracks/0OFknyqxmSQ4...,0.9999998922
81,Astronaut In The Ocean,3Ofmpyhv5UAQ70mENzB277,https://api.spotify.com/v1/tracks/3Ofmpyhv5UAQ...,0.9999998904


In [48]:
recommendations.drop_duplicates(inplace=True)

Index(['name', 'id', 'link', 'cosin'], dtype='object')

In [50]:
recommendations = recommendations.reset_index().drop('index', axis=1)
recommendations

,name,id,link,cosin
0,"Raataan Lambiyan (From ""Shershaah"")",2rOnSn2piaqLAlYjtfUBlY,https://api.spotify.com/v1/tracks/2rOnSn2piaqL...,1.1097642117
1,Raataan Lambiyan,1stiSonuKkZqhI1o9nZ9MT,https://api.spotify.com/v1/tracks/1stiSonuKkZq...,1.1097642117
2,Dil Galti Kar Baitha Hai,7HPM13Xek5XoRWxRzHfvkB,https://api.spotify.com/v1/tracks/7HPM13Xek5Xo...,1.1097642067
3,Without Me,6FZDfxM3a3UCqtzo5pxSLZ,https://api.spotify.com/v1/tracks/6FZDfxM3a3UC...,0.9999999999
4,Feather,2Zo1PcszsT9WQ0ANntJbID,https://api.spotify.com/v1/tracks/2Zo1PcszsT9W...,0.9999999999
...,...,...,...,...
128,Manwa Laage,3H43T5swYywvcdCBFiDgW6,https://api.spotify.com/v1/tracks/3H43T5swYywv...,0.9999999333
129,Mast Magan,3uL1IBFhg52VcQqOwAG01E,https://api.spotify.com/v1/tracks/3uL1IBFhg52V...,0.9999999331
130,Un Poco Loco,0OFknyqxmSQ42SoKxWVTok,https://api.spotify.com/v1/tracks/0OFknyqxmSQ4...,0.9999998922
131,Astronaut In The Ocean,3Ofmpyhv5UAQ70mENzB277,https://api.spotify.com/v1/tracks/3Ofmpyhv5UAQ...,0.9999998904


In [46]:
recommendations.head(50)

,name,id,link,cosin
0,"Raataan Lambiyan (From ""Shershaah"")",2rOnSn2piaqLAlYjtfUBlY,https://api.spotify.com/v1/tracks/2rOnSn2piaqL...,1.1097642117
1,Raataan Lambiyan,1stiSonuKkZqhI1o9nZ9MT,https://api.spotify.com/v1/tracks/1stiSonuKkZq...,1.1097642117
2,Dil Galti Kar Baitha Hai,7HPM13Xek5XoRWxRzHfvkB,https://api.spotify.com/v1/tracks/7HPM13Xek5Xo...,1.1097642067
3,Without Me,6FZDfxM3a3UCqtzo5pxSLZ,https://api.spotify.com/v1/tracks/6FZDfxM3a3UC...,0.9999999999
4,Feather,2Zo1PcszsT9WQ0ANntJbID,https://api.spotify.com/v1/tracks/2Zo1PcszsT9W...,0.9999999999
5,What Else Can I Do?,3XoYqtiWHhsk59frZupImG,https://api.spotify.com/v1/tracks/3XoYqtiWHhsk...,0.9999999999
6,"Strangers Like Me - From ""Tarzan""/Soundtrack V...",5Zb6JxSEcjsUNrINcAyAYJ,https://api.spotify.com/v1/tracks/5Zb6JxSEcjsU...,0.9999999997
7,Trampoline (with ZAYN),1iQDltZqI7BXnHrFy4Qo1k,https://api.spotify.com/v1/tracks/1iQDltZqI7BX...,0.9999999996
8,Mood (feat. iann dior),4jPy3l0RUwlUI9T5XHBW2m,https://api.spotify.com/v1/tracks/4jPy3l0RUwlU...,0.9999999995
9,Bones,54ipXppHLA8U4yqpOFTUhr,https://api.spotify.com/v1/tracks/54ipXppHLA8U...,0.9999999995


In [47]:
sp.start_playback(uris=['spotify:track:1stiSonuKkZqhI1o9nZ9MT'])

ReadTimeout: HTTPSConnectionPool(host='api.spotify.com', port=443): Read timed out. (read timeout=5)

THE END 🙂